In [1]:
import pandas as pd 

In [3]:
# Load cleaned data
data3=pd.read_csv('cleaned_data2_iotid23.csv')


In [4]:
data3.shape

(1230402, 35)

In [5]:
for column in data3.columns:
    print(f"Value counts for column: {column}")
    print(data3[column].value_counts())
    print("\n" + "=" * 50 + "\n")  # Separator for better readability


Value counts for column: Protocol
Protocol
6     815865
0     236808
17    177729
Name: count, dtype: int64


Value counts for column: Flow Duration
Flow Duration
2           8234
3           7602
1           6407
4           2212
9           1689
            ... 
18583617       1
10510736       1
21998999       1
9112427        1
146262         1
Name: count, Length: 447786, dtype: int64


Value counts for column: Total Fwd Packet
Total Fwd Packet
1       722482
2       318451
3        52570
4        42838
5         9989
         ...  
1799         1
3661         1
3626         1
3604         1
508          1
Name: count, Length: 1143, dtype: int64


Value counts for column: Total Length of Fwd Packet
Total Length of Fwd Packet
0.0        797832
2.0         69810
28.0        42304
6.0         24678
24.0        20122
            ...  
14944.0         1
75307.0         1
19257.0         1
40474.0         1
20340.0         1
Name: count, Length: 10480, dtype: int64


Value counts for col